In [1]:
%pylab inline
from scipy.optimize import curve_fit
import pandas as pd
import seaborn as sns
import numpy as np
import scipy 
import matplotlib.pyplot as plt
import matplotlib.style as style
import itertools
from sklearn.linear_model import LogisticRegression
import sys

import warnings
warnings.filterwarnings('ignore')


Populating the interactive namespace from numpy and matplotlib


In [2]:
df_home = pd.read_excel('df_team_with_past_HST_home.xlsx')
df_away = pd.read_excel('df_team_with_past_HST_away.xlsx')

In [3]:
df_home.head()

,Unnamed: 0,Day,Month,Year,HomeTeam,AwayTeam,FTHG,FTAG,HTGDIFF,ATGDIFF,...,HTGDIFF_15,HTGDIFF_16,HTGDIFF_17,HTGDIFF_18,HTGDIFF_19,HTGDIFF_20,HTGDIFF_21,HTGDIFF_22,HTGDIFF_23,HST_1
0,133,15,12,2019,Wolfsburg,M'gladbach,0,0,0,0,...,-3,2,0,1,-1,-2,0,-2,0,6
1,134,15,12,2019,Schalke 04,Ein Frankfurt,0,0,0,0,...,-2,1,-1,-1,3,2,-2,1,-2,8
2,127,14,12,2019,Bayern Munich,Werder Bremen,0,0,0,0,...,2,3,1,3,0,0,-3,0,2,11
3,128,14,12,2019,Hertha,Freiburg,0,0,0,0,...,0,0,1,0,-3,0,2,2,1,3
4,129,14,12,2019,Mainz,Dortmund,0,0,0,0,...,0,0,-1,1,-1,0,0,1,1,8


In [4]:
df_away.head()

,Unnamed: 0,Day,Month,Year,HomeTeam,AwayTeam,FTHG,FTAG,HTGDIFF,ATGDIFF,...,ATGDIFF_15,ATGDIFF_16,ATGDIFF_17,ATGDIFF_18,ATGDIFF_19,ATGDIFF_20,ATGDIFF_21,ATGDIFF_22,ATGDIFF_23,AST_1
0,133,15,12,2019,Wolfsburg,M'gladbach,0,0,0,0,...,-1,0,-2,2,-2,3,0,-2,0,2
1,134,15,12,2019,Schalke 04,Ein Frankfurt,0,0,0,0,...,0,-1,2,3,0,1,-2,-2,2,4
2,127,14,12,2019,Bayern Munich,Werder Bremen,0,0,0,0,...,0,1,-1,-1,0,-1,2,-1,1,3
3,128,14,12,2019,Hertha,Freiburg,0,0,0,0,...,0,-2,1,-2,-2,0,0,-3,2,5
4,129,14,12,2019,Mainz,Dortmund,0,0,0,0,...,0,1,-1,1,1,1,4,2,0,5


In [5]:
df_away.columns

Index(['Unnamed: 0', 'Day', 'Month', 'Year', 'HomeTeam', 'AwayTeam', 'FTHG',
       'FTAG', 'HTGDIFF', 'ATGDIFF', 'AVGATGDIFF', 'AVGFTAG', 'HS', 'AS',
       'HST', 'AST', 'HC', 'AC', 'HF', 'AF', 'HY', 'AY', 'HR', 'AR',
       'ATGDIFF_1', 'ATGDIFF_2', 'ATGDIFF_3', 'ATGDIFF_4', 'ATGDIFF_5',
       'ATGDIFF_6', 'ATGDIFF_7', 'ATGDIFF_8', 'ATGDIFF_ATGDIFF_9',
       'ATGDIFF_10', 'ATGDIFF_11', 'ATGDIFF_12', 'ATGDIFF_13', 'ATGDIFF_14',
       'ATGDIFF_15', 'ATGDIFF_16', 'ATGDIFF_17', 'ATGDIFF_18', 'ATGDIFF_19',
       'ATGDIFF_20', 'ATGDIFF_21', 'ATGDIFF_22', 'ATGDIFF_23', 'AST_1'],
      dtype='object')

In [6]:
cols_to_use = df_home.columns.difference(df_away.columns)

In [7]:
cols_to_use

Index(['AVGFTHG', 'AVGHTGDIFF', 'HST_1', 'HTGDIFF_1', 'HTGDIFF_10',
       'HTGDIFF_11', 'HTGDIFF_12', 'HTGDIFF_13', 'HTGDIFF_14', 'HTGDIFF_15',
       'HTGDIFF_16', 'HTGDIFF_17', 'HTGDIFF_18', 'HTGDIFF_19', 'HTGDIFF_2',
       'HTGDIFF_20', 'HTGDIFF_21', 'HTGDIFF_22', 'HTGDIFF_23', 'HTGDIFF_3',
       'HTGDIFF_4', 'HTGDIFF_5', 'HTGDIFF_6', 'HTGDIFF_7', 'HTGDIFF_8',
       'HTGDIFF_9'],
      dtype='object')

In [8]:
df_both = pd.merge(df_away, df_home[cols_to_use], left_index=True, right_index=True, how='outer')

In [9]:
df_both['W'], df_both['D'], df_both['L'] = [0, 0, 0]

In [10]:
del df_both['Unnamed: 0']

In [11]:
df_both = df_both.reindex(columns = ['Day', 'Month', 'Year', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG',
                                              'W',
       'D', 'L',
       'HTGDIFF', 'ATGDIFF', 'AVGATGDIFF', 'AVGFTAG', 'HS', 'AS', 'HST', 'AST',
       'HC', 'AC', 'HF', 'AF', 'HY', 'AY', 'HR', 'AR', 'ATGDIFF_1',
       'ATGDIFF_2', 'ATGDIFF_3', 'ATGDIFF_4', 'ATGDIFF_5', 'ATGDIFF_6',
       'ATGDIFF_7', 'ATGDIFF_8', 'ATGDIFF_ATGDIFF_9', 'ATGDIFF_10',
       'ATGDIFF_11', 'ATGDIFF_12', 'ATGDIFF_13', 'ATGDIFF_14', 'ATGDIFF_15',
       'ATGDIFF_16', 'ATGDIFF_17', 'ATGDIFF_18', 'ATGDIFF_19', 'ATGDIFF_20',
       'ATGDIFF_21', 'ATGDIFF_22', 'ATGDIFF_23', 'AST_1', 'AVGFTHG',
       'AVGHTGDIFF', 'HST_1', 'HTGDIFF_1', 'HTGDIFF_10', 'HTGDIFF_11',
       'HTGDIFF_12', 'HTGDIFF_13', 'HTGDIFF_14', 'HTGDIFF_15', 'HTGDIFF_16',
       'HTGDIFF_17', 'HTGDIFF_18', 'HTGDIFF_19', 'HTGDIFF_2', 'HTGDIFF_20',
       'HTGDIFF_21', 'HTGDIFF_22', 'HTGDIFF_23', 'HTGDIFF_3', 'HTGDIFF_4',
       'HTGDIFF_5', 'HTGDIFF_6', 'HTGDIFF_7', 'HTGDIFF_8', 'HTGDIFF_9'])

In [12]:
df_both.to_excel('both.xlsx')